In [9]:
import requests 
import pandas as pd
import time
import json
from datetime import date,datetime,timedelta

In [10]:
#Declare file config
file_path = '../credentials/config.json'

#Open and read Json file
with open(file_path, "r") as file:
    config_data = json.load(file)

In [11]:
current_date = date.today()
current_date = current_date.strftime('%d/%m/%Y')
print("Current Date:", current_date)


Current Date: 22/10/2025


In [12]:
url = "https://fc-data.ssi.com.vn/api/v2/Market/AccessToken"

payload = {
    "consumerID": config_data["consumerID"],
    "consumerSecret": config_data["consumerSecret"]
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
if response.status_code == 200:
    access_token = response.json()['data'].get("accessToken")
else:
    print("Failed to get access token. Status Code:", response.status_code)
    print("Response Body:", response.text)
    exit()

In [13]:
start_date = (date.today() - timedelta(days = 1))
end_date = (date.today() - timedelta(days = 1))

In [14]:
print(start_date)
print(end_date)

2025-10-21
2025-10-21


In [15]:
# Replace with the token you obtained
url = "https://fc-data.ssi.com.vn/api/v2/Market/DailyOhlc"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

df = pd.DataFrame()
for data_date in pd.date_range(start=start_date, end=end_date):
    data_date = data_date.strftime('%d/%m/%Y')
    
    for index in range(1, 10):
        params = {
            "fromDate": data_date,
            "toDate": data_date,
            "pageIndex": index,
            "pageSize": 1000
        }
        response = requests.get(url, headers=headers, params=params)
        data = response.json().get('data', [])
        df_response = pd.DataFrame(data)
        df = pd.concat([df, df_response], ignore_index=True)
        time.sleep(1)
        if len(df_response) == 0:
            break

In [16]:
print(df)

             Symbol Market TradingDate  Time     Open     High      Low  \
0             HNX30   None  21/10/2025  None   566.26   575.87   553.59   
1          HNXIndex   None  21/10/2025  None   263.39   266.99   259.40   
2     HNXUpcomIndex   None  21/10/2025  None   108.48   110.02   108.38   
3             VN100   None  21/10/2025  None  1809.51  1849.50  1780.75   
4              VN30   None  21/10/2025  None  1882.66  1933.22  1862.51   
...             ...    ...         ...   ...      ...      ...      ...   
1250            XMD  UPCOM  21/10/2025  None     6800     7000     6200   
1251            XMP  UPCOM  21/10/2025  None    15000    15300    15000   
1252            XPH  UPCOM  21/10/2025  None    14700    14700    14600   
1253            YBM   HOSE  21/10/2025  None    14000    14350    13700   
1254            YEG   HOSE  21/10/2025  None    12050    12750    12050   

        Close      Volume           Value  
0      567.58   106435315   2845263174700  
1      264.